In [197]:
%config InlineBackend.figure_formats = ['svg']
import quimb
import quimb.tensor as qtn
import numpy as np
import matplotlib.pyplot as plt

from functions_numpy import *

In [198]:
# Define Dl string of observables
def Ommd(n, sigma):
    """
    Building the observable Dl for a given sigma and n. The standard name for the indexes are k1, k2, ..., kn, b1, b2, ..., bn.
    """
    # Set parameters:
    n = 9             # number of qubits per half
    L = 2 * n         # total number of sites

    Dl_list = []
    # Questa è la sommatoria di tutti i Dl con il loro coefficiente coef1
    for l in range(1, n+1):
        p_sigma = (1 - np.exp(-1/(2*sigma)))/2
        coef = p_sigma**l * (1-p_sigma)**(n-l)

        A_l = A(n, l)

        # Building D2l
        mpo_list = []
        for i in A_l:
            site1 = i
            site2 = site1 + n

            # Define operators:
            Z = np.array([[1, 0],
                        [0, -1]])
            I = np.eye(2)

            # Build MPO tensors: each tensor is shaped (1, 1, 2, 2)
            mpo_tensors = []
            for site in range(L):
                # Choose Z on the designated sites, I elsewhere:
                op = Z if site in site1 or site in site2 else I
                tensor = op.reshape(1, 2, 2) if site in [0, L - 1] else op.reshape(1, 1, 2, 2)
                mpo_tensors.append(tensor)

            # Create the MPO. Here, 'sites' and 'L' help label the tensor network.
            mpo = qtn.MatrixProductOperator(
                mpo_tensors,
                sites=range(L),
                L=L,
                shape='lrud'
            )
            mpo_list.append(mpo)

        # Sum all the MPOs
        Dl = mpo_list[0]
        for i in mpo_list[1:]:
            Dl = Dl.add_MPO(i)
        
        Dl_list.append(coef*Dl)

    O = Dl_list[0]
    for i in Dl_list[1:]:
        Dl = Dl.add_MPO(i)
    
    return Dl

In [199]:
def MMD(x, y,Ommd, sigma, number_open_index, bond_dimension):
    """
    samples and target are two Matrix Product States.
    """

    x /= x.H @ x
    y /= y.H @ y
    rename_dict = {f'k{i}': f'k{i+number_open_index}' for i in range(number_open_index)}
    y.reindex_(rename_dict)

    """
    Building the MMD MPO. The default open indexes are k1, k2, ..., kn, b1, b2, ..., bn.
    Then we can contract the MPO with the MPS and the bitstring state to get the loss function.
    """

    x.add_tag('x')
    y.add_tag('y')
    Ommd.add_tag('Ommd')

    loss = x & Ommd & y

    #qtn.drawing.draw_tn(loss, color={"x": "blue", "Ommd": "red", "y": "green"}, figsize=(30, 30), show_inds=True)

    # Here we should do the trace but we have a MPS, what happends then??
    loss = loss @ loss.H
    

    return loss


dimension of dataset: 14

In [200]:
L = 9
D = 8
sigma = 0.09

# create a random MPS as our initial target to optimize
psi = qtn.MPS_rand_state(L, bond_dim=D)
Ommd = Ommd(L, sigma)
dataset = get_bars_and_stripes(3)

In [201]:
MPS_dataset = []
for data in dataset:
    MPS_dataset.append(qtn.MPS_computational_state(data))


In [202]:
def loss_fn(psi,dataset,Ommd):
    loss = 0
    #for data in dataset:
    data = psi.copy()
    loss += MMD(psi, data, Ommd, sigma, L, D)
    #loss = loss / len(dataset)
    return loss


In [203]:
tnopt = qtn.TNOptimizer(
    # the tensor network we want to optimize
    psi,
    # the functions specfying the loss and normalization
    loss_fn=loss_fn,
    #norm_fn=norm_fn,
    # we specify constants so that the arguments can be converted
    # to the  desired autodiff backend automatically
    loss_constants={"dataset": MPS_dataset, "Ommd": Ommd},
    # the underlying algorithm to use for the optimization
    # 'l-bfgs-b' is the default and often good for fast initial progress
    optimizer="adam",
    # which gradient computation backend to use
    autodiff_backend="numpy",
)

In [204]:
psi_opt = tnopt.optimize(1)

+1.656370153747 [best: +1.656370153747] : : 2it [00:02,  1.08s/it]                     
